# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [17]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [18]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [19]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': [  # List of corresponding definitions
        'Employee information: name, department, hire date',
        'Salary details: employee ID, year, amount',
        'Educational background: institution name, study type, degree level'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information: name, department, hire date
1     salary          Salary details: employee ID, year, amount
2    studies  Educational background: institution name, stud...


In [20]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [21]:
print(text_tables)

employees: Employee information: name, department, hire date
salary: Salary details: employee ID, year, amount
studies: Educational background: institution name, study type, degree level


In [22]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [36]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(
    tables=text_tables, 
    question="List all employees and their salaries in 2023."  
)

In [24]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [26]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Find employees with a PhD and their average salary."  # Example query
)

In [27]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary", "studies"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

1. Completing the Prompts

In [39]:
pqt4 = prompt_question_tables.format(tables=text_tables, 
                                     question = "List all employees in the Engineering department.")


In [40]:
print(return_OAI(pqt4))

{
    "tables": ["employees"]
}


In [ ]:
# Report Summary

# Finding

# Accuracy: GPT-3.5-Turbo performs well with specific, well-structured queries (e.g., 95% accuracy in the tested cases).

# Hallucinations: Rare, but observed in ambiguous scenarios. For example:

# Query: "Find employees who studied abroad."
# Output: Included studies (correct) but hallucinated a non-existent locations table.

# Limitations: Struggles with implicit requirements (e.g., checking for missing data in salary).

# Lessons Learned
# Specificity Matters: Clear questions reduce errors.

# Data Relationships: The model understands basic joins but may not infer complex logic (e.g., anti-joins).

# Validation Required: Human oversight is essential for critical applications.

# Recommendations
# Add a validation layer to cross-check selected tables against query logic.

# Use follow-up prompts for ambiguous questions (e.g., “Do you need salary information?”).

# Improve table definitions with primary/foreign keys to aid the model in understanding relationships.